In [5]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder, StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.decomposition import PCA

In [6]:
df = pd.read_csv("UNSW_NB15_training-set.csv")

In [7]:
df.describe()

,id,dur,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,...,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,label
count,175341.000000,175341.000000,175341.000000,175341.000000,1.753410e+05,1.753410e+05,1.753410e+05,175341.000000,175341.000000,1.753410e+05,...,175341.000000,175341.000000,175341.000000,175341.000000,175341.000000,175341.000000,175341.000000,175341.000000,175341.000000,175341.000000
mean,87671.000000,1.359389,20.298664,18.969591,8.844844e+03,1.492892e+04,9.540619e+04,179.546997,79.609567,7.345403e+07,...,5.383538,4.206255,8.729881,0.014948,0.014948,0.133066,6.955789,9.100758,0.015752,0.680622
std,50616.731112,6.480249,136.887597,110.258271,1.747656e+05,1.436542e+05,1.654010e+05,102.940011,110.506863,1.883574e+08,...,8.047104,5.783585,10.956186,0.126048,0.126048,0.701208,8.321493,10.756952,0.124516,0.466237
min,1.000000,0.000000,1.000000,0.000000,2.800000e+01,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,...,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000
25%,43836.000000,0.000008,2.000000,0.000000,1.140000e+02,0.000000e+00,3.278614e+01,62.000000,0.000000,1.305334e+04,...,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,2.000000,2.000000,0.000000,0.000000
50%,87671.000000,0.001582,2.000000,2.000000,4.300000e+02,1.640000e+02,3.225807e+03,254.000000,29.000000,8.796748e+05,...,1.000000,1.000000,3.000000,0.000000,0.000000,0.000000,3.000000,4.000000,0.000000,1.000000
75%,131506.000000,0.668069,12.000000,10.000000,1.418000e+03,1.102000e+03,1.250000e+05,254.000000,252.000000,8.888889e+07,...,5.000000,3.000000,12.000000,0.000000,0.000000,0.000000,9.000000,12.000000,0.000000,1.000000
max,175341.000000,59.999989,9616.000000,10974.000000,1.296523e+07,1.465555e+07,1.000000e+06,255.000000,254.000000,5.988000e+09,...,51.000000,46.000000,65.000000,4.000000,4.000000,30.000000,60.000000,62.000000,1.000000,1.000000


In [8]:
def preprocess(df: pd.DataFrame):
    # drop doops
    # drop nulls
    # labelencode
    # normalize
    df = df.copy(deep=True)
    df.drop_duplicates(inplace=True)
    df.dropna(inplace=True)
    
    X, Y = df.drop('label', axis=1), df['label']
    lc = LabelEncoder()
    sc = StandardScaler()

    for col in X.columns:
        if X[col].dtype == object:
            X[col] = lc.fit_transform(X[col])

    df = pd.DataFrame(sc.fit_transform(X), columns=X.columns, index=X.index)
    df['label'] = Y
    return df

In [9]:
def feature_selection(df, threshold):
    # remove features that have multi collinearity above thold.
    df = df.copy(deep=True)

    corr_matrix = df.drop('label', axis=1).corr().abs()

    upr_triangle = np.triu(corr_matrix, k=1)

    return [col for i, col in enumerate(corr_matrix.columns) if any(upr_triangle[i, :] > threshold)]

In [10]:
def sampling(df):
    df = df.copy(deep=True)
    X, Y = df.drop('label', axis=1), df['label']

    smote = SMOTE(random_state=42, k_neighbors=6)
    X, Y = smote.fit_resample(X, Y)

    df = pd.DataFrame(X, columns=X.columns, index=X.index)
    df['label'] = Y

    return df

In [11]:
def pca_transform(df, pca):
    df = df.copy(deep=True)

    X, Y = df.drop('label', axis=1), df['label']

    trX = pca.transform(X)

    df = pd.DataFrame(trX, columns=[f'PC{i+1}' for i in range(trX.shape[1])], index=X.index)
    df['label'] = Y
    return df

In [12]:
df = preprocess(df)

In [13]:
df

,id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,-1.732041,-0.191029,0.151809,-0.702307,-0.409218,-0.104456,-0.135769,-0.049134,-0.102726,-0.576371,...,-0.554373,-0.705529,-0.118590,-0.118590,-0.189768,-0.715714,-0.753074,-0.126508,0.848024,0
1,-1.732021,-0.109485,0.151809,-0.702307,-0.409218,-0.046014,0.172599,-0.046410,0.188544,-0.576345,...,-0.554373,-0.614256,-0.118590,-0.118590,-0.189768,-0.715714,-0.288257,-0.126508,0.848024,0
2,-1.732001,0.040699,0.151809,-0.702307,-0.409218,-0.089845,-0.026933,-0.048527,-0.012133,-0.576734,...,-0.554373,-0.522983,-0.118590,-0.118590,-0.189768,-0.595543,-0.288257,-0.126508,0.848024,0
3,-1.731982,0.049729,0.151809,0.599130,-0.409218,-0.060624,-0.063212,-0.047016,-0.098563,-0.576737,...,-0.554373,-0.522983,7.814915,7.814915,-0.189768,-0.595543,-0.753074,-0.126508,0.848024,0
4,-1.731962,-0.140417,0.151809,-0.702307,-0.409218,-0.075235,-0.117630,-0.047554,-0.102057,-0.576617,...,-0.554373,2.854115,-0.118590,-0.118590,-0.189768,-0.595543,2.779535,-0.126508,0.848024,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175336,1.731962,-0.209773,0.420235,0.165317,0.742936,-0.133677,-0.172047,-0.049958,-0.103923,0.094951,...,1.520470,1.393748,-0.118590,-0.118590,-0.189768,2.048221,1.385084,-0.126508,0.210364,1
175337,1.731982,-0.131728,0.151809,-0.702307,-0.409218,-0.075235,-0.099490,-0.047062,-0.101459,-0.576616,...,-0.554373,-0.614256,-0.118590,-0.118590,-0.189768,-0.715714,-0.753074,-0.126508,2.123345,1
175338,1.732001,-0.209773,0.420235,0.165317,0.742936,-0.133677,-0.172047,-0.049958,-0.103923,0.094951,...,-0.208566,0.389746,-0.118590,-0.118590,-0.189768,-0.475371,0.269523,-0.126508,0.210364,1
175339,1.732021,-0.209773,0.420235,0.165317,0.742936,-0.133677,-0.172047,-0.049958,-0.103923,0.094951,...,1.693374,1.941386,-0.118590,-0.118590,-0.189768,2.769248,1.942865,-0.126508,0.210364,1


In [14]:
high_corr_features = feature_selection(df, 0.8)

In [15]:
high_corr_features

['spkts',
 'dpkts',
 'sbytes',
 'dbytes',
 'dttl',
 'sinpkt',
 'swin',
 'tcprtt',
 'ct_srv_src',
 'ct_dst_ltm',
 'ct_src_dport_ltm',
 'ct_dst_sport_ltm',
 'ct_dst_src_ltm',
 'is_ftp_login']

In [16]:
df.drop(columns=high_corr_features, inplace=True)

In [17]:
df = sampling(df)

In [18]:
df

,id,dur,proto,service,state,rate,sttl,sload,dload,sloss,...,trans_depth,response_body_len,ct_state_ttl,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,-1.732041,-0.191029,0.151809,-0.702307,-0.409218,-0.576371,0.703839,-0.389897,-0.273700,-0.075040,...,-0.136415,-0.039557,-1.366486,-0.118590,-0.189768,-0.715714,-0.753074,-0.126508,0.848024,0
1,-1.732021,-0.109485,0.151809,-0.702307,-0.409218,-0.576345,-1.141901,-0.389928,-0.069233,-0.044739,...,-0.136415,-0.039557,-0.318711,-0.118590,-0.189768,-0.715714,-0.288257,-0.126508,0.848024,0
2,-1.732001,0.040699,0.151809,-0.702307,-0.409218,-0.576734,-1.141901,-0.389964,-0.252044,-0.059890,...,-0.136415,-0.039557,-0.318711,-0.118590,-0.189768,-0.595543,-0.288257,-0.126508,0.848024,0
3,-1.731982,0.049729,0.151809,0.599130,-0.409218,-0.576737,-1.141901,-0.389958,-0.275821,-0.059890,...,-0.136415,-0.039557,-0.318711,7.814915,-0.189768,-0.595543,-0.753074,-0.126508,0.848024,0
4,-1.731962,-0.140417,0.151809,-0.702307,-0.409218,-0.576617,0.723268,-0.389927,-0.275561,-0.044739,...,-0.136415,-0.039557,-0.318711,-0.118590,-0.189768,-0.595543,2.779535,-0.126508,0.848024,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238677,-1.260766,0.749012,0.151809,-0.702307,-0.409218,-0.576665,-1.443048,-0.389945,-0.226154,0.031013,...,-0.136415,-0.039557,-1.366486,-0.118590,-0.189768,-0.549885,-0.506255,-0.126508,0.848024,0
238678,-0.861817,-0.128238,0.151809,-0.702307,-0.409218,-0.576642,0.723268,-0.389932,-0.275763,-0.044739,...,-0.136415,-0.039557,-0.318711,-0.118590,-0.189768,-0.715714,-0.284188,-0.126508,0.848024,0
238679,0.277653,-0.175018,0.151809,-0.702307,-2.713526,-0.576631,-1.141901,-0.389817,-0.276576,-0.044739,...,-0.136415,-0.039557,1.776839,-0.118590,-0.189768,-0.259692,-0.103546,-0.126508,0.848024,0
238680,-1.094044,0.032858,0.151809,0.599130,-0.409218,-0.576345,-1.443048,-0.389881,-0.268151,0.091614,...,-0.136415,-0.039557,-1.366486,7.814915,-0.189768,-0.235029,-0.693022,-0.126508,0.848024,0


In [19]:
pca = PCA(n_components=6).fit(df.drop('label', axis=1))
df = pca_transform(df, pca)

In [20]:
df

,PC1,PC2,PC3,PC4,PC5,PC6,label
0,1.063917,-0.341830,-0.151708,-0.170984,-0.958117,-0.613448,0
1,2.764191,-1.227381,0.063195,0.531181,1.151272,-0.429514,0
2,3.000737,0.808467,0.712435,0.827495,0.854733,-0.429295,0
3,1.671501,-0.000221,0.468636,-0.006421,-1.149580,1.314883,0
4,0.603309,1.068339,-0.833571,-0.157897,-0.212779,-1.468532,0
...,...,...,...,...,...,...,...
238677,3.837860,-0.115007,0.977613,1.594204,1.457699,0.365186,0
238678,1.459659,1.905259,-0.601955,-1.018699,-0.017859,-1.613009,0
238679,1.093591,1.930260,-0.408913,-1.340917,-0.878681,-1.175456,0
238680,1.566417,-0.537880,0.114920,-0.256938,-1.380733,1.340386,0


In [21]:
from sklearn.model_selection import train_test_split

In [22]:
X, Y = df.drop('label', axis=1), df['label']

In [23]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.20)

In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import classification_report

In [25]:
sample_model = LogisticRegression()
sample_model.fit(x_train, y_train)
sample_model_preds = sample_model.predict(x_test)

print(classification_report(y_test, sample_model_preds))

              precision    recall  f1-score   support

           0       0.92      0.87      0.90     23898
           1       0.88      0.93      0.90     23839

    accuracy                           0.90     47737
   macro avg       0.90      0.90      0.90     47737
weighted avg       0.90      0.90      0.90     47737



In [26]:
models = [LogisticRegression(), KNeighborsClassifier(), SVC(), DecisionTreeClassifier(), RandomForestClassifier(), GradientBoostingClassifier(), AdaBoostClassifier()]

In [27]:
for model in models:
    print(type(model).__name__)
    score = cross_val_score(model, x_train, y_train, cv=2, scoring='accuracy').mean()
    print(score)

LogisticRegression
0.8993794026898165
KNeighborsClassifier
0.9824137823181169
SVC
0.9549084829248924
DecisionTreeClassifier
0.9727356033407452
RandomForestClassifier
0.9841001329973267
GradientBoostingClassifier
0.9517662145233438
AdaBoostClassifier
0.9212600423123987


In [28]:
# X = df.drop('label', axis=1)
# y = df['label']

# Define base models
model1 = LogisticRegression()
model2 = KNeighborsClassifier()
model3 = SVC()
model4 = DecisionTreeClassifier()
model5 = RandomForestClassifier()
model6 = AdaBoostClassifier()
model7 = GradientBoostingClassifier()

# Define ensemble model
ensemble = VotingClassifier(estimators=[('lr', model1), ('knn', model2), ('svc', model3), ('dt', model4), ('rf', model5), ('ab', model6), ('gb', model7)], voting='hard')

# Define hyperparameters to tune
params = {
    'lr__C': [1], 
    'rf__n_estimators': [10], 
    'svc__C': [1], 
    'knn__n_neighbors': [3], 
    'dt__max_depth': [None],
    'ab__n_estimators': [5],
    'gb__n_estimators': [5],
    'gb__learning_rate': [0.1]
}

# Perform grid search
grid = GridSearchCV(estimator=ensemble, param_grid=params, cv=5)
grid.fit(x_test, y_test)

# Print best parameters
print(grid.best_params_)

{'ab__n_estimators': 5, 'dt__max_depth': None, 'gb__learning_rate': 0.1, 'gb__n_estimators': 5, 'knn__n_neighbors': 3, 'lr__C': 1, 'rf__n_estimators': 10, 'svc__C': 1}


In [30]:
import time

In [31]:
chosen_model_1 = KNeighborsClassifier(n_neighbors=3)
chosen_model_2 = RandomForestClassifier(n_estimators=3)
chosen_model_3 = DecisionTreeClassifier(max_depth=None)
chosen_model_4 = GradientBoostingClassifier(learning_rate=0.1, n_estimators=5)

# Define ensemble model
start_time = time.time()

ensemble = VotingClassifier(estimators=[('knn', chosen_model_1), ('rf', chosen_model_2), ('dt', chosen_model_3), ('gb', chosen_model_4)], voting='hard')
ensemble.fit(x_train, y_train)
ensemble_preds = ensemble.predict(x_test)

end_time = time.time()

print(f"\nExecution took {end_time - start_time} seconds.")
print("\nEnsemble Model Performance:")
print(classification_report(y_test, ensemble_preds))


Execution took 5.794909238815308 seconds.

Ensemble Model Performance:
              precision    recall  f1-score   support

           0       0.98      0.99      0.98     23898
           1       0.99      0.98      0.98     23839

    accuracy                           0.98     47737
   macro avg       0.98      0.98      0.98     47737
weighted avg       0.98      0.98      0.98     47737



In [35]:
sklearn.model_selection??

Type:        module
String form: <module 'sklearn.model_selection' from '/home/karthikmsd/.local/lib/python3.10/site-packages/sklearn/model_selection/__init__.py'>
File:        ~/.local/lib/python3.10/site-packages/sklearn/model_selection/__init__.py
Source:     
import typing

from ._split import BaseCrossValidator
from ._split import BaseShuffleSplit
from ._split import KFold
from ._split import GroupKFold
from ._split import StratifiedKFold
from ._split import TimeSeriesSplit
from ._split import LeaveOneGroupOut
from ._split import LeaveOneOut
from ._split import LeavePGroupsOut
from ._split import LeavePOut
from ._split import RepeatedKFold
from ._split import RepeatedStratifiedKFold
from ._split import ShuffleSplit
from ._split import GroupShuffleSplit
from ._split import StratifiedShuffleSplit
from ._split import StratifiedGroupKFold
from ._split import PredefinedSplit
from ._split import train_test_split
from ._split import check_cv

from ._validation import cross_val_score
from

In [33]:
import sklearn